### Some fun with the λ-calculus

A helper function to get names of λ-terms that we define

In [ ]:
def lambda_name(lmb):
    # Return the name of the lambda function based on it's value
    # OR
    # Return church-encoded number based on the number of applications

    for name, value in globals().items():
        # this just some python magic
        # 
        # search current scope and 
        # checks if `lmb` object already defined
        if value is lmb:
            return name
    
    # other case -- assume that lmb is a number, and 
    # by number definition find it's value
    # lambda s: lambda z: s(s(z))
    # s became "plus one func" and z -- 0    
    return f"c{lmb(lambda x: x + 1)(0)}"

In [ ]:
a = "a"
b = "b"
c = "c"

lambda_name(b)

Identity function: **λx. x**

In [ ]:
id = lambda x: x

In [ ]:
print(id(3))
print(id("Hi"))
# (λx.x) (λx.x)
print(lambda_name(id(id)))
# (λx.x) (λx.x) (λx.x) (λx.x)
print(lambda_name(id(id)(id)(id)))
# (λx.x) (λx.x) (λx.x) a
print(lambda_name(id(id)(id)(a)))

Funtions of multiple arguments: **λx.λy.x** and **λx.λy.y**

In [ ]:
first = lambda x: lambda y: x
second = lambda x: lambda y: y

In [ ]:
# (λx.λy.x) a b
print(first(a)(b))

# (λx.λy.y) a b
print(second(a)(b))


Some examples

In [ ]:
# (λx.λy.x) (λx.λy.x) (λx.λy.x) a b
print(first(first)(first)(a)(b))

# (λx.λy.x) (λx.x) (λx.λy.y) b
# first (λx.x) (λx.λy.y) b
# (first (λx.x) (λx.λy.y)) b
# (λx.x) b
# b
print((lambda x: lambda y: x)(lambda x: x)(lambda x: lambda y: y)(b))


#### Booleans

In [ ]:
tru = first
fls = second
# if-then-else
# λb.λt.λf.b t f
ite = lambda b: lambda t: lambda f: b(t)(f)

In [ ]:
print(ite(tru)(a)(b))
print(ite(fls)(a)(b))

Logical AND

In [ ]:
# x AND y = ?
# To answer this question, let's consider two cases:
# 1) tru AND y = y
# 2) fls AND y = fls
#
# a AND b = if a then b else fls
AND = lambda x: lambda y: ite(x)(y)(fls)

print(lambda_name(AND(tru)(tru)))
print(lambda_name(AND(fls)(tru)))
print(lambda_name(AND(tru)(fls)))
print(lambda_name(AND(fls)(fls)))

print()

# Alternatively, we can see that
# 1) tru(y)(fls) = y
# 2) fls(y)(fls) = fls
#
# x AND y = x y fls
AND = lambda x: lambda y: x(y)(fls)

# Note: we basically opened the definition of ite:
# ite x y fls
# = (λb.λt.λf.b t f) x y fls
# = x y fls

print(lambda_name(AND(tru)(tru)))
print(lambda_name(AND(fls)(tru)))
print(lambda_name(AND(tru)(fls)))
print(lambda_name(AND(fls)(fls)))

Logical NOT

In [ ]:
# not tru = fls
# not fls = tru
#
# Notice:
# tru fls tru = fls
# fls fls tru = tru
#
# λb.b fls tru
NOT = lambda b: b(fls)(tru)

print(lambda_name(NOT(fls)))
print(lambda_name(NOT(tru)))

Exercises on logical operators

In [ ]:
# Exercise 1
OR = lambda a: lambda b: 0
# Exercise 2
XOR = lambda a: lambda b: 0

#### Natural Numbers

In [ ]:
# Church encoding of natural numbers:
# We can encode a natural number as a function of two arguments:
# 1) Some unary operator
# 2) Some value
#
# Natural number `n` is then a function which applies the operator to the value n times:
#
# 0 := λs.λz.z
# 1 := λs.λz.s z
# 2 := λs.λz.s (s z)
# 3 := λs.λz.s (s (s z))
c0 = lambda s: lambda z: z          # applies `s` to `z` 0 times
c1 = lambda s: lambda z: s(z)       # applies `s` to `z` 1 time
c2 = lambda s: lambda z: s(s(z))    # applies `s` to `z` 2 times
c3 = lambda s: lambda z: s(s(s(z))) # applies `s` to `z` 3 times

In [ ]:
lambda_name(lambda s: lambda z: s(s(z)))

Some operations on church-encoded natural numbers

In [ ]:
# Increment
#
# Number `n` applies `s` to `z` n times. Then, we apply `s` one more time:
# λn.λs.λz.s (n s z)
inc = lambda n: lambda s: lambda z: s(n(s)(z))

print(lambda_name(inc(inc(c2))))

# Check if the number is zero
#
# We can make a function that ignores its argument and return false.
# If we apply this function more than once, we get false.
# If we apply this function zero times, we are left with the initial value:
# λn. n (λx.fls) tru
isZero = lambda n: n(lambda x: fls)(tru)

print(lambda_name(isZero(c0)))
print(lambda_name(isZero(c1)))
print(lambda_name(isZero(c2)))


Addition (exercise)

In [ ]:
# Exercise 3
add = lambda n: lambda m: lambda s: lambda z: 0
print(lambda_name(add(c3)(c2)))

Multiplication

In [ ]:
# 1) λn.λm. n (λk. add k m) c0
mult = lambda n: lambda m: n(lambda k: add(k)(m))(c0)

print(lambda_name(mult(c3)(c2)))
print(lambda_name(mult(c3)(c3)))

# 1.2) λn.λm. n (add m) c0
mult = lambda n: lambda m: n(add(m))(c0)

print(lambda_name(mult(c3)(c2)))
print(lambda_name(mult(c3)(c3)))

# 2) λn.λm.λs.λz. n (m s) z
mult = lambda n: lambda m: lambda s: lambda z: n (m(s)) (z)

print(lambda_name(mult(c3)(c2)))
print(lambda_name(mult(c3)(c3)))

Exponentiation (exercise)

In [ ]:
exp = lambda n: lambda m: 0

#### Pairs

In [ ]:
# λx.λy.λf.f x y
pair = lambda x: lambda y: lambda f: f(x)(y)

# λp.p first
# λp.p second
fst = lambda p: p(first)
snd = lambda p: p(second)

In [ ]:
# fst (pair a b)
# = (λp.p first) (pair a b)
# = (λp.p first) ((λx.λy.λf.f x y) a b)
# = (λp.p first) (λf.f a b)
# = (λf.f a b) first
# = first a b
# = a
print(lambda_name(fst(pair(a)(b))))

